## 📌 1. Présentation du contexte de l'article

### 🧠 Problème considéré

L’article **“Stochastic Variational Inference”** de Hoffman, Blei, Wang et Paisley (2013) s’attaque à un **problème fondamental** en apprentissage bayésien : comment faire de l’**inférence approchée dans des modèles probabilistes complexes**, quand le volume de données est énorme.

Ces modèles incluent des structures hiérarchiques avec :
- **Variables latentes locales**, propres à chaque donnée (ex. : proportions de sujets d’un document dans LDA),
- **Paramètres globaux**, partagés entre toutes les données (ex. : sujets globaux dans LDA).

L’enjeu est de pouvoir **calculer la distribution postérieure** de ces variables, ce qui est généralement **intractable en pratique**.

---

### 📦 Type de données concernées

L’article se concentre sur des **données massives** où chaque observation suit une structure similaire :
- Documents textuels (comme dans LDA),
- Séries temporelles,
- Graphes,
- Données biologiques, etc.

Le principal exemple étudié est le modèle **LDA (Latent Dirichlet Allocation)** appliqué à un corpus scientifique contenant **plus d’un million de documents**.

---

### ❌ Limites des approches existantes

Les approches classiques d’inférence bayésienne souffrent de **graves limitations** à grande échelle :

- **MCMC** (Monte Carlo par chaînes de Markov) : précis mais **trop lent**, surtout pour des millions d’exemples.
- **Inférence variationnelle standard** (batch VI) : transforme l’inférence en un problème d’optimisation, plus rapide que MCMC, mais encore **trop coûteuse** :
  - Nécessite de **parcourir l’intégralité des données** à chaque itération.
  - Consomme beaucoup de mémoire et est **difficilement parallélisable**.

---

### 🎯 Objectif de l'article

Proposer une méthode :
- **Scalable** : qui fonctionne à l’échelle de millions de données,
- **Efficace** : basée sur des **mini-batchs**,
- Théoriquement solide : convergence prouvée,
- Appliquée à des modèles de la **famille exponentielle conjuguée** (comme LDA).

C’est dans ce contexte qu’intervient la méthode **Stochastic Variational Inference (SVI)**, qui combine inférence variationnelle + descente de gradient stochastique pour un apprentissage **rapide et itératif**.

---


## 🧠 2. Méthode proposée : Stochastic Variational Inference (SVI)

---

### 2.1 🎯 Objectif du cadre variationnel

Dans les modèles bayésiens hiérarchiques comme LDA, l’objectif est de calculer la distribution postérieure :

$$
p(\beta, z \mid x) = \frac{p(x, z, \beta)}{p(x)}
$$

Mais le dénominateur $p(x) = \int p(x, z, \beta)\,dz\,d\beta$ est incomputable dans les cas réels.

On approxime cette postérieure par une distribution factorisée :

$$
q(z, \beta) = q(\beta \mid \lambda) \prod_{n=1}^N q(z_n \mid \phi_n)
$$

---

### 2.2 💡 Inférence variationnelle classique

On minimise la divergence de Kullback-Leibler :

$$
\mathrm{KL}(q(z, \beta) \parallel p(z, \beta \mid x)) = \mathbb{E}_q\left[\log \frac{q(z, \beta)}{p(z, \beta \mid x)}\right]
$$

Ce qui revient à maximiser l’ELBO :

$$
\mathcal{L}(\lambda, \phi) = \mathbb{E}_q[\log p(x, z, \beta)] - \mathbb{E}_q[\log q(z, \beta)]
$$

> *“Variational inference turns the problem of posterior inference into one of optimization.”*  
> *(Page 2, colonne de droite)*

---

### 2.3 ⚠️ Limites de l’approche batch

Chaque mise à jour des paramètres globaux $\lambda$ dépend de toutes les données → trop coûteux pour de grands jeux.

> *“In massive data sets, traditional variational inference is computationally intractable.”*  
> *(Page 2, colonne de droite)*

---

### 2.4 💡 Idée de SVI : optimiser l’ELBO par sous-échantillons

SVI remplace le gradient exact de l’ELBO par une estimation issue d’un sous-échantillon.

On réécrit :

$$
\mathcal{L}(\lambda) = \sum_{i=1}^N \mathcal{L}_i(\lambda)
$$

Et on met à jour $\lambda$ par descente stochastique :

$$
\lambda^{(t)} = \lambda^{(t-1)} + \rho_t \cdot \hat{\nabla} \mathcal{L}_i(\lambda)
$$

---

### 2.5 🔢 Structure des modèles conjugués

Le modèle doit être de la forme exponentielle conjuguée :

- Prior :

$$
p(\beta) = h(\beta)\exp(\eta^\top t(\beta) - a(\eta))
$$

- Likelihood :

$$
p(x, z \mid \beta) = f(x, z)\exp(t(\beta)^\top u(x, z))
$$

Cela permet des mises à jour analytiques via les statistiques suffisantes.

> *“Our method applies to a large class of models with conjugate exponential families.”*  
> *(Page 3, colonne de droite)*

---

### 2.6 📐 Mise à jour naturelle des paramètres

Mise à jour avec le **gradient naturel** (plus stable) :

$$
\lambda^{(t)} = (1 - \rho_t) \lambda^{(t-1)} + \rho_t \hat{\lambda}_i
$$

> *“We adapt stochastic optimization to variational inference using the natural gradient of the variational objective.”*  
> *(Page 5, colonne de gauche)*

---

### 2.7 ⚙️ Taux d’apprentissage

$$
\rho_t = (t + \tau)^{-\kappa}
$$

avec $\kappa \in (0.5, 1]$, $\tau > 0$

> *“These conditions ensure the stochastic algorithm converges to a local optimum of the ELBO.”*  
> *(Appendix A)*

---

### 2.8 📘 Application à LDA

- Variables locales : $\theta_d$, $z_{dn}$
- Paramètres globaux : $\beta_k$

SVI permet de :

- Tirer un document
- Optimiser ses variables locales
- Mettre à jour les distributions de sujets $\beta_k$

> Voir Figure 2, Page 13 pour l’algorithme complet

---

### ✅ Avantages de SVI

| Aspect       | Avantage                                        |
|--------------|-------------------------------------------------|
| Vitesse      | Mises à jour locales rapides                    |
| Mémoire      | Pas besoin de charger tout le dataset           |
| Scalabilité  | Compatible avec des millions de documents       |
| Théorique    | Convergence démontrée sous conditions           |
| Modulaire    | Fonctionne avec d'autres modèles hiérarchiques |

---
